In [1]:
import pandas as pd
path = "D:/temp_files/datasets/spaceship_titanic/"
train_full = pd.read_csv(path + "train_full.csv")
test_X = pd.read_csv(path + "test_full.csv")
test = pd.read_csv(path + "test.csv")
from sklearn.model_selection import train_test_split
train_full_X = train_full.drop(axis=1,labels='Transported')
train_full_y = pd.DataFrame(data=train_full['Transported'],columns=['Transported'])
train_X, valid_X, train_y, valid_y = train_test_split(train_full_X, train_full_y, test_size=0.1)
from sklearn.metrics import mean_absolute_error

In [3]:
from tensorflow import keras
mlp_model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[train_X.shape[1]]),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
mlp_model.compile(loss='sparse_categorical_crossentropy',optimizer=keras.optimizers.SGD(learning_rate=1e-3),metrics=['accuracy'])
history = mlp_model.fit(train_X, train_y, epochs=50,validation_data=(valid_X, valid_y))
import numpy as np
predictions = np.argmax(mlp_model.predict(test_X), axis=1)

output = pd.DataFrame({'PassengerId': test.PassengerId, 'Transported': predictions})
output['Transported'] = output['Transported'].astype('bool')
output.to_csv('submission_mlp.csv', index=False)


134/134 [==============================] - 0s 3ms/step


In [4]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score
accuracy = []
recall = []
fold_no =[]
def model(X,Y,folds):
    skf = StratifiedKFold(n_splits=3,shuffle=True,random_state=0)
    for fold,(train_idx,test_idx) in enumerate(skf.split(X,Y)):
        train_X,val_X = X.iloc[train_idx],X.iloc[test_idx]
        train_y,val_y = Y.iloc[train_idx],Y.iloc[test_idx]
        
        clf = RandomForestClassifier(n_estimators=100,max_depth=12)
        
        clf.fit(train_X,train_y)
        pred = clf.predict(val_X)
        
        accuracy.append(round(accuracy_score(val_y,pred),4))
        recall.append(round(recall_score(val_y,pred),4))
        fold_no.append(fold+1)
        
    
    
    result_df = pd.DataFrame({'Accuracy' : accuracy,'Recall' : recall},index=fold_no)
    
    return result_df,clf

In [5]:
result_df,model = model(train_full_X,train_full_y,5)
result_df

C:\Users\DD\AppData\Local\Temp\ipykernel_11640\2873382259.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(train_X,train_y)
C:\Users\DD\AppData\Local\Temp\ipykernel_11640\2873382259.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(train_X,train_y)
C:\Users\DD\AppData\Local\Temp\ipykernel_11640\2873382259.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(train_X,train_y)


,Accuracy,Recall
1,0.7540,0.6669
2,0.7505,0.6774
3,0.7553,0.6628


In [7]:
predictions = model.predict(test_X)
sub = pd.read_csv(path + 'sample_submission.csv')
t = pd.read_csv(path + 'test.csv')
sub['PassengerId'] = t['PassengerId']
sub['Transported'] = predictions
sub['Transported'] = sub['Transported'].astype(bool)
sub.to_csv('my_submission.csv',index=False)